在上一章节，我们使用传统机器学习算法来解决了文本分类问题，从本章开始我们将尝试使用深度学习方法。

## **Task4 基于深度学习的文本分类1-fastText**

与传统机器学习不同，深度学习既提供特征提取功能，也可以完成分类的功能。从本章开始我们将学习如何使用深度学习来完成文本表示。

### **学习目标**

- 学习FastText的使用和基础原理
- 学会使用验证集进行调参

### **文本表示方法 Part2**

#### **现有文本表示方法的缺陷**

在上一章节，我们介绍几种文本表示方法：

- One-hot
- Bag of Words
- N-gram
- TF-IDF

也通过sklean进行了相应的实践，相信你也有了初步的认知。但上述方法都或多或少存在一定的问题：转换得到的向量维度很高，需要较长的训练实践；没有考虑单词与单词之间的关系，只是进行了统计。

与这些表示方法不同，深度学习也可以用于文本表示，还可以将其映射到一个低纬空间。其中比较典型的例子有：FastText、Word2Vec和Bert。在本章我们将介绍FastText，将在后面的内容介绍Word2Vec和Bert。

#### **FastText**

FastText是一种典型的深度学习词向量的表示方法，它非常简单通过Embedding层将单词映射到稠密空间，然后将句子中所有的单词在Embedding空间中进行平均，进而完成分类操作。

所以FastText是一个三层的神经网络，输入层、隐含层和输出层。



<div align=center><img src="http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/1095279501877/1594909720411_wruLzMgC7N.jpg" width="50%" height="50%"></div>

下图是使用keras实现的FastText网络结构：


<div align=center><img src="http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/1095279501877/1594909821168_NvB7c98dSc.jpg" width="500" height="500"></div>


FastText在文本分类任务上，是优于TF-IDF的：

- FastText用单词的Embedding叠加获得的文档向量，将相似的句子分为一类
- FastText学习到的Embedding空间维度比较低，可以快速进行训练

如果想深度学习，可以参考论文：

Bag of Tricks for Efficient Text Classification, https://arxiv.org/abs/1607.01759

### **基于FastText的文本分类**

FastText可以快速的在CPU上进行训练，最好的实践方法就是官方开源的版本：
https://github.com/facebookresearch/fastText/tree/master/python

- pip安装

```
pip install fasttext
```

- 源码安装

```
git clone https://github.com/facebookresearch/fastText.git
cd fastText
sudo pip install .
```

* 两种安装方法都可以安装，如果你是初学者可以优先考虑使用pip安装。     
    * pip 直接安装时报错.
    * 改为下载后pip安装. 参考: https://blog.csdn.net/yyy430/article/details/88534319

In [1]:
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv(r'data\train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')

In [33]:
from datetime import datetime
start =datetime.now()
print(f'开始时间:{start}')

import fasttext
# 参数
lr=1.0;dim=100;wordNgrams=3;verbose=2;minCount=1;epoch=50;loss='softmax'
model = fasttext.train_supervised('train.csv', lr=lr, dim=dim, wordNgrams=wordNgrams, verbose=verbose, minCount=minCount, epoch=epoch, loss=loss)

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]

end = datetime.now()
print(f'结束时间:{end}')
print(f"用时:{str(end-start)[:7]}. lr={lr}; dim={dim}; wordNgrams={wordNgrams}; verbose={verbose}; minCount={minCount}; epoch={epoch}; loss='{loss}'==> f1_score={f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro')}")

# lr=1.0;wordNgrams=2;verbose=2;minCount=1;epoch=25; loss='hs',==> f1_score=0.8169674577657406  七分钟
# lr=0.1;wordNgrams=2;verbose=2;minCount=1;epoch=25; loss='hs',==> f1_score=0.6444837436918328  48秒
# lr=0.1;wordNgrams=2;verbose=2;minCount=1;epoch=50; loss='hs',==> f1_score=0.7629222252290507  一分半
# lr=0.1;wordNgrams=2;verbose=2;minCount=1;epoch=100;loss='hs',==> f1_score=0.8089565290736888  三分钟
# lr=1.0;wordNgrams=2;verbose=2;minCount=1;epoch=100;loss='hs',==> f1_score=0.8236167304069028  九分钟
# lr=1.0;wordNgrams=2;verbose=2;minCount=1;epoch=100;loss='hs',==> f1_score=0.8232324296578212
#用时:0:00:23. lr=1.0, wordNgrams=2, verbose=2, minCount=1, epoch=10, loss='hs'==> f1_score=0.8057418720787898
#用时:0:00:52. lr=0.5, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss='hs'==> f1_score=0.8138595789035314
#用时:0:00:52. lr=1.0, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss='hs'==> f1_score=0.8163094536071093
#用时:0:00:53. lr=1.5, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss='hs'==> f1_score=0.8240406623753461
#用时:0:00:53. lr=2.5, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss='hs'==> f1_score=0.8250824474725055
#用时:0:01:33. lr=2.5, wordNgrams=3, verbose=2, minCount=1, epoch=25, loss='hs'==> f1_score=0.8329760152666325
#用时:0:01:34. lr=2.5, dim=100, wordNgrams=3, verbose=2, minCount=1, epoch=25, loss='hs'==> f1_score=0.8334896465010527
#用时:0:03:40. lr=2.5, dim=300, wordNgrams=3, verbose=2, minCount=1, epoch=25, loss='hs'==> f1_score=0.8305045198663006
#用时:0:02:40. lr=2.5; dim=200; wordNgrams=3; verbose=2; minCount=1; epoch=25; loss='hs'==> f1_score=0.8293814479044828
#用时:0:02:00. lr=2.5; dim=150; wordNgrams=3; verbose=2; minCount=1; epoch=25; loss='hs'==> f1_score=0.8288513811399753
#用时:0:01:25. lr=2.5; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=25; loss='hs'==> f1_score=0.8331790300210206
#用时:0:01:26. lr=2.5; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=25; loss='softmax'==> f1_score=0.8792997247236528
#用时:0:02:51. lr=0.5; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=50; loss='softmax'==> f1_score=0.8793241507976178
#用时:0:02:47. lr=1.0; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=50; loss='softmax'==> f1_score=0.8797862764526768


开始时间:2020-07-27 21:23:08.360732
结束时间:2020-07-27 21:25:55.962318
用时:0:02:47. lr=1.0; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=50; loss='softmax'==> f1_score=0.8797862764526768


此时数据量比较小得分为0.82，当不断增加训练集数量时，FastText的精度也会不断增加, 5w条训练样本时，验证集得分可以到0.89-0.90左右。

In [35]:
# 转换为FastText需要的格式--读取5w条数据
train_df = pd.read_csv(r'data\train_set.csv', sep='\t', nrows=50000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].iloc[:-5000].to_csv('train5w.csv', index=None, header=None, sep='\t')

from datetime import datetime
start =datetime.now()
print(f'开始时间:{start}')

import fasttext
# 参数
lr=1.0;dim=100;wordNgrams=3;verbose=2;minCount=1;epoch=50;loss='hs'
model = fasttext.train_supervised('train5w.csv', lr=lr, dim=dim, wordNgrams=wordNgrams, verbose=verbose, minCount=minCount, epoch=epoch, loss=loss)

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]

end = datetime.now()
print(f'结束时间:{end}')
print(f"用时:{str(end-start)[:7]}. lr={lr}; dim={dim}; wordNgrams={wordNgrams}; verbose={verbose}; minCount={minCount}; epoch={epoch}; loss='{loss}'==> f1_score={f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro')}")
#用时:0:13:55. lr=1.0; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=50; loss='softmax'==> f1_score=0.924038383183665
#用时:0:13:55. lr=1.0; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=50; loss='softmax'==> f1_score=0.924038383183665


开始时间:2020-07-27 21:53:38.624417
结束时间:2020-07-27 22:07:25.844731
用时:0:13:47. lr=1.0; dim=100; wordNgrams=3; verbose=2; minCount=1; epoch=50; loss='hs'==> f1_score=0.8973484830301076


In [3]:
fasttext.train_supervised??

Signature: fasttext.train_supervised(*kargs, **kwargs)
Source:   
def train_supervised(*kargs, **kwargs):
    """
    Train a supervised model and return a model object.

    input must be a filepath. The input text does not need to be tokenized
    as per the tokenize function, but it must be preprocessed and encoded
    as UTF-8. You might want to consult standard preprocessing scripts such
    as tokenizer.perl mentioned here: http://www.statmt.org/wmt07/baseline.html

    The input file must must contain at least one label per line. For an
    example consult the example datasets which are part of the fastText
    repository such as the dataset pulled by classification-example.sh.
    """
    supervised_default = unsupervised_default.copy()
    supervised_default.update({
        'lr': 0.1,
        'minCount': 1,
        'minn': 0,
        'maxn': 0,
        'loss': "softmax",
        'model': "supervised"
    })

    arg_names = ['input', 'lr', 'dim', 'ws', 'epoch', 'minCount',
  


### **如何使用验证集调参**

在使用TF-IDF和FastText中，有一些模型的参数需要选择，这些参数会在一定程度上影响模型的精度，那么如何选择这些参数呢？
      

- 通过阅读文档，要弄清楚这些参数的大致含义，那些参数会增加模型的复杂度
- 通过在验证集上进行验证模型精度，找到模型在是否过拟合还是欠拟合

In [ ]:
一些方法:　　　　
对数据进行预处理--如:识别并去掉标点 ；     
更改迭代次数 (通过选项-epoch，标准范围 [5 - 50]) ；       
更改学习速率 (使用选项 -lr，标准范围 [0.1 - 1.0])；            
使用 word n-grams (使用选项 -wordNgrams，标准范围 [1 - 5])。 



<div align=center><img src="http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/1095279501877/1594909879453_RrvunJz6cT.jpg" width="50%" height="50%"></div>

此时数据量比较小得分为0.82，当不断增加训练集数量时，FastText的精度也会不断增加, 5w条训练样本时，验证集得分可以到0.89-0.90左右。

### **如何使用验证集调参**

在使用TF-IDF和FastText中，有一些模型的参数需要选择，这些参数会在一定程度上影响模型的精度，那么如何选择这些参数呢？

- 通过阅读文档，要弄清楚这些参数的大致含义，那些参数会增加模型的复杂度
- 通过在验证集上进行验证模型精度，找到模型在是否过拟合还是欠拟合



<div align=center><img src="http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/1095279501877/1594909879453_RrvunJz6cT.jpg" width="50%" height="50%"></div>

这里我们使用10折交叉验证，每折使用9/10的数据进行训练，剩余1/10作为验证集检验模型的效果。这里需要注意每折的划分必须保证标签的分布与整个数据集的分布一致。

```
label2id = {}
for i in range(total):
    label = str(all_labels[i])
    if label not in label2id:
        label2id[label] = [i]
    else:
        label2id[label].append(i)
```

通过10折划分，我们一共得到了10份分布一致的数据，索引分别为0到9，每次通过将一份数据作为验证集，剩余数据作为训练集，获得了所有数据的10种分割。不失一般性，我们选择最后一份完成剩余的实验，即索引为9的一份做为验证集，索引为1-8的作为训练集，然后基于验证集的结果调整超参数，使得模型性能更优。

### **本章小结**

本章介绍了FastText的原理和基础使用，并进行相应的实践。然后介绍了通过10折交叉验证划分数据集。

### **本章作业**

- 阅读FastText的文档，尝试修改参数，得到更好的分数
- 基于验证集的结果调整超参数，使得模型性能更优



**关于Datawhale：**

> Datawhale是一个专注于数据科学与AI领域的开源组织，汇集了众多领域院校和知名企业的优秀学习者，聚合了一群有开源精神和探索精神的团队成员。Datawhale 以“for the learner，和学习者一起成长”为愿景，鼓励真实地展现自我、开放包容、互信互助、敢于试错和勇于担当。同时 Datawhale 用开源的理念去探索开源内容、开源学习和开源方案，赋能人才培养，助力人才成长，建立起人与人，人与知识，人与企业和人与未来的联结。

本次新闻文本分类学习，专题知识将在天池分享，详情可关注Datawhale：

 ![](http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/1095279172547/1584432602983_kAxAvgQpG2.jpg)